## 1. Изучите структуру модуля src
#### Done

## 2. Перенесите функции prefilter_items и postfilter_items в модуль src.utils.py
#### уже были перенесены

## 3. Реализуйте функции get_similar_items_recommendation, get_similar_users_recommendation (они разбирались на вебинаре) и переместите в src.utils.py.  
## postfilter можно пока оставить пустой (или как-то бороться с товаром 999999) - на Ваше усмотрение.  
## В prefilter нужно реализовать закомментированные ограничения. 
#### а зачем нам эту функции и в utils, и в recommendation.py? Наверное, лучше сразу разместить это в собственном классе, поскольку при его инициализации мы можем посчитать, например, топ покупок по всем юзерам

In [1]:
def prefilter_items(data_train, item_features, departpments_):
    # Оставим только 5000 самых популярных товаров
    popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
    popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
    top = popularity.sort_values('n_sold', ascending=False).head(5100).item_id.tolist()
    #добавим, чтобы не потерять юзеров
    data_train.loc[~data_train['item_id'].isin(top), 'item_id'] = 999999 
    
    # Уберем самые популярные 
    data_train.loc[data_train['item_id'].isin(top[:100]), 'item_id'] = 999999 
    
    # Уберем самые непопулряные 
    # не совсем понял, уберем самые неполпулярные из популярных? Разделить вне топ-5000 от хвоста топ-5000?
    
    # Уберем товары, которые не продавались за последние 12 месяцев
    timeline = data_train['week_no'].max() - 54
    date_filter = data_train['item_id'].loc[data_train['week_no'] >= timeline].unique()
    data_train = data_train.loc[~data_train['item_id'].isin(date_filter), 'item_id'].shape
    # Уберем не интересные для рекоммендаций категории (department)
    departments_filter = item_features.loc[~item_features['department'].isin(departpments_), 'item_id'].unique()
    data_train = data_train.loc[~data_train['item_id'].isin(deps_filter)]
    
    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб. 
    data_train = data_train[data_train.sales_value > 1.5]
    
    # Уберем слишком дорогие товарыs
    price_filter = data_train['sales_value'].quantile(0.90)
    data_train = data_train[data_train.sales_value < price_filter]
    # ...
    
    return data_train

## 4. Создайте модуль src.recommenders.py. Напишите код для класса ниже и положите его в src.recommenders.py  
## get_similar_items_recommendation и get_similar_users_recommendation - реализуйте их, как делалось на вебинаре

In [ ]:
# def prefilter_items(data_train, item_features, departpments_):
#     # Оставим только 5000 самых популярных товаров
#     popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
#     popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
#     top = popularity.sort_values('n_sold', ascending=False).head(5100).item_id.tolist()
#     #добавим, чтобы не потерять юзеров
#     data_train.loc[~data_train['item_id'].isin(top), 'item_id'] = 999999 
    
#     # Уберем самые популярные 
#     data_train.loc[data_train['item_id'].isin(top[:100]), 'item_id'] = 999999 
    
#     # Уберем самые непопулряные 
#     # не совсем понял, уберем самые неполпулярные из популярных? Разделить вне топ-5000 от хвоста топ-5000?
    
#     # Уберем товары, которые не продавались за последние 12 месяцев
#     timeline = data_train['week_no'].max() - 54
#     date_filter = data_train['item_id'].loc[data_train['week_no'] >= timeline].unique()
#     data_train = data_train.loc[data_train['item_id'].isin(date_filter)]
    
#     # Уберем не интересные для рекоммендаций категории (department)
    
    
#     departments_filter = item_features.loc[~item_features['department'].isin(departpments_), 'item_id'].unique()
#     data_train = data_train.loc[~data_train['item_id'].isin(departments_filter)]
    
     
#     # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб. 
#     data_train = data_train[data_train.sales_value > 1.5]
    
#     # Уберем слишком дорогие товарыs
#     price_filter = data_train['sales_value'].quantile(0.90)
#     data_train = data_train[data_train.sales_value < price_filter]
#     # ...
    
#     return data_train

## 5. Проверьте, что все модули корректно импортируются

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
item_features = pd.read_csv('product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [5]:
from src.recommenders import MainRecommender
from src.utils import prefilter_items
from src.metrics import precision_at_k

In [6]:
deps_to_drop = ['PASTRY', 'GROCERY']

In [7]:
data_prefilter = prefilter_items(data_train, item_features, deps_to_drop)

C:\Users\Pavel\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [8]:
#model = MainRecommender(data_prefilter)

In [9]:
# import pandas as pd
# import numpy as np

# # Для работы с матрицами
# from scipy.sparse import csr_matrix

# # Матричная факторизация
# from implicit.als import AlternatingLeastSquares
# from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
# from implicit.nearest_neighbours import bm25_weight, tfidf_weight


# class MainRecommender:
#     """Рекоммендации, которые можно получить из ALS
#     Input
#     -----
#     user_item_matrix: pd.DataFrame
#         Матрица взаимодействий user-item
#     """
    
#     def __init__(self, data, weighting=True):
        
#         #your_code. Это не обязательная часть. Но если вам удобно что-либо посчитать тут - можно это сделать
#         self.popularity = data.groupby(['user_id','item_id'])['quantity'].count().reset_index()
#         self.popularity.sort_values('quantity', ascending=False, inplace=True)
            
#         self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
#         self.id_to_itemid, self.id_to_userid,\
#             self.itemid_to_id, self.userid_to_id = self.prepare_dicts(self.user_item_matrix)
        
#         if weighting:
#             self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T 
        
#         self.model = self.fit(self.user_item_matrix)
#         self.own_recommender = self.fit_own_recommender(self.user_item_matrix)
     
#     @staticmethod
#     def prepare_matrix(data):
#         user_item_matrix = pd.pivot_table(data, 
#                                   index='user_id', columns='item_id', 
#                                   values='quantity', # Можно пробоват ьдругие варианты
#                                   aggfunc='count', 
#                                   fill_value=0
#                                  )

#         user_item_matrix = user_item_matrix.astype(float)
        
#         return user_item_matrix
    
#     @staticmethod
#     def prepare_dicts(user_item_matrix):
#         """Подготавливает вспомогательные словари"""

#         userids = user_item_matrix.index.values
#         itemids = user_item_matrix.columns.values

#         matrix_userids = np.arange(len(userids))
#         matrix_itemids = np.arange(len(itemids))

#         id_to_itemid = dict(zip(matrix_itemids, itemids))
#         id_to_userid = dict(zip(matrix_userids, userids))

#         itemid_to_id = dict(zip(itemids, matrix_itemids))
#         userid_to_id = dict(zip(userids, matrix_userids))

#         return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id
     
#     @staticmethod
#     def fit_own_recommender(user_item_matrix):
#         """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
    
#         own_recommender = ItemItemRecommender(K=1, num_threads=4)
#         own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())
        
#         return own_recommender
    
#     @staticmethod
#     def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
#         """Обучает ALS"""
        
#         model = AlternatingLeastSquares(factors=n_factors, 
#                                              regularization=regularization,
#                                              iterations=iterations,  
#                                              num_threads=num_threads)
#         model.fit(csr_matrix(user_item_matrix).T.tocsr())
        
#         return model
    
 
#     def get_rec(self, x):
#         recs = self.model.similar_items(self.itemid_to_id[x], N=2)
#         top_rec = recs[1][0]
#         return self.id_to_itemid[top_rec]

#     def get_similar_items_recommendation(self, user, N=5):
        
#         top_n = self.popularity[self.popularity['user_id']==user]
               
#         top_n = top_n.head(N)
        
#         top_n['similar_item'] = top_n['item_id'].apply(lambda x: self.get_rec(x))
#         #print(top_n)
#         res = top_n['similar_item']
    
        
#         #assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
#         return list(res)
    
#     def get_similar_users_recommendation(self, user, N=5):
#         """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
#         sparse_user_item=csr_matrix(self.user_item_matrix).T.tocsr()
        
#         res = [self.id_to_itemid[rec[0]] for rec in 
#                     self.model.recommend(userid=self.userid_to_id[user], 
#                                     user_items=sparse_user_item,   # на вход user-item matrix
#                                     N=N, 
#                                     filter_already_liked_items=False, 
#                                     #filter_items=[itemid_to_id[999999]],  # !!! 
#                                     recalculate_user=True)]
        
    
#         # your_code

#         assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
#         return res

In [10]:
model = MainRecommender(data_prefilter)

In [11]:
model.get_similar_items_recommendation(2375)

[1076056, 9526411, 978354, 8020234, 1070497]

In [12]:
model.get_similar_users_recommendation(2375)

[1080884, 936634, 918733, 902192, 1128647]